In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec 22 20:24:24 2020

@author: Onolab2
"""

import pyvisa
import pprint
import os
import datetime
import pandas as pd
import numpy as np
import math
from MDSplus import *
rm=pyvisa.ResourceManager() #リソースマネージャ
devname=('GPIB0::1::INSTR','GPIB0::29::INSTR')
pprint.pprint(devname)#読み込まれたデバイスの表示　
dev_maxnum=len(devname)#デバイス数
print(dev_maxnum)#デバイス数の表示
inst=[0 for i in range(dev_maxnum)]#デバイス数分の接続先のリスト化

def ConnectDevice(i_d):#接続する関数
    inst[i_d] = rm.open_resource(devname[i_d])
    inst[i_d].timeout = 10000
    return print("CONNECTED "+devname[i_d])

def StartDev(i_d):
    inst[i_d].write(":START")
    return print("START "+str(i_d))

def StopDev(i_d):
    inst[i_d].write("STOP")
    return print("STOP "+str(i_d))

def GetWaveSignal_1(i_d,ch):#波形の取り込み   
    inst[i_d].write("COMM:HEAD 0")
    clockFreq=float(inst[i_d].query("WAV:SRAT?"))
    clockPeriod = 1./clockFreq
    trace_str = "WAVEFORM:TRACe "+str(int(ch))
    print(trace_str)
    inst[i_d].write(trace_str)   #読み込み先を設定
    inst[i_d].write("WAVEFORM:FORMAT WORD") #BYTE形式で読み込み
    #print(inst[i_d-1].query("WAVEFORM:SIGN?"))  #符号ariのBYTE形式
    inst[i_d].write("WAVEFORM:START 0")    #読み込み開始地点の設定
    inst[i_d].write("WAVEFORM:END "+inst[i_d-1].query("WAVEFORM:LENGTH?"))
    
    vdiv = float(inst[i_d].query(":WAV:RANG?"))
    ofst = float(inst[i_d].query(":WAV:OFFSET?"))
    #dp=inst[i_d-1].query("WAVEFORM:LENGTH?")
    #print(dp)
    raw = inst[i_d].query_binary_values("WAVEFORM:SEND?",datatype="h")#,data_points=10032)#生データを出力
    #raw = inst[i_d].query_ascii_values("WAVEFORM:SEND?")#,data_points=10032)#生データを出力

    print(len(raw))
    
    data = []
    for num in raw:
        #data.append(int(num))
        data.append(int(num)*vdiv*8/64000+ofst) #count
        #data.append(int(num)*vdiv*8/250+ofst) #電圧値=(BYTE 数値×電圧レンジ)/12.5+オフセット値        #data.append(int(num)/32*vdiv+ofst if num < 128 else (int(num)-256)/32*vdiv+ofst) #電圧値=(BYTE 数値×電圧レンジ)/12.5+オフセット値
    return data

####MAIN################################################################################################
    
ConnectDevice(0)
ConnectDevice(1)


StartDev(0)
StartDev(1)

now = datetime.datetime.now()

today=now.strftime('%y%m%d')

if not os.path.exists('C:/Users/Onolab2/Desktop/toroidal mode results/'+today):#ディレクトリがなかったら
   os.makedirs('C:/Users/Onolab2/Desktop/toroidal mode results/'+today)#作成したいフォルダ名を作成 Xにはold_koalaの場所を入れる
   print("directory created")


print(inst[0].query("*CAL?"))
print(inst[1].query("*CAL?"))
 
#Caribrationall()

inst[0].write("disp:cle")
inst[1].write("disp:cle")

print("\nWaiting for trigger!!!!")

shot=input('shot number: ')

#os.chdir(today)

StopDev(0)
StopDev(1)

filename = 'C:/Users/Onolab2/Desktop/toroidal mode results/'+today+'/toroidal_mode' + now.strftime('%Y%m%d_%H%M%S_No') +shot+ '.csv'

tt = ["time(us)","ch1","ch2","ch3","ch4","ch5","ch6","ch7","ch8","ch9","ch10","ch11","ch12","ch13","ch14","ch15","ch16","ch17","ch18","ch19","ch20","ch21","ch22","ch23","ch24"]
ff=list()
fff = list()
#時間軸を書く
i = 0
fff.append(i)
for pp in range (10019):
    i += 1
    pp = i*0.1
    fff.append(pp)
   
ff.append(fff)

l=[1,2,3,4,5,6,7,8]
ll=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]   #オシロで使うチャンネルを指定


for shoooot in ll:
  ff.append(GetWaveSignal_1(1,shoooot))

for shooot in l:
    ff.append(GetWaveSignal_1(0,shooot))
    
    

    
ff = pd.DataFrame(ff).T.values.tolist()
df = pd.DataFrame(ff,columns = tt)
df.to_csv(filename,index=False)

print("\n"+filename+" ok")

os.chdir(os.path.dirname(os.path.abspath("__file__")))

ModuleNotFoundError: No module named 'pyvisa'